In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO
import gradio as gr
import numpy as np, cv2, torch, time
from IPython.display import Javascript, display, clear_output
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model = YOLO('yolov8n-pose.pt')

In [5]:
# 0 nose, 1 l_eye, 2 r_eye, 3 l_ear, 4 r_ear,
# 5 l_shoulder, 6 r_shoulder, 7 l_elbow, 8 r_elbow,
# 9 l_wrist, 10 r_wrist, 11 l_hip, 12 r_hip,
# 13 l_knee, 14 r_knee, 15 l_ankle, 16 r_ankle

def angle(a, b, c):
    a, b, c = np.array(a, dtype=np.float32), np.array(b, dtype=np.float32), np.array(c, dtype=np.float32)
    ba, bc = a - b, c - b
    denom = (np.linalg.norm(ba) * np.linalg.norm(bc)) + 1e-6
    cosang = float(np.clip(np.dot(ba, bc) / denom, -1.0, 1.0))
    return float(np.degrees(np.arccos(cosang)))

def euclid(p1, p2):
    p1, p2 = np.array(p1, dtype=np.float32), np.array(p2, dtype=np.float32)
    return float(np.linalg.norm(p1 - p2))

def get_kpt_xy(kpts, idx):
    return (float(kpts[idx, 0]), float(kpts[idx, 1]))

def is_visible(kpts, idx, thr=0.25):
    return float(kpts[idx, 2]) >= thr


In [6]:
def draw_text(img, text, x, y, scale=0.9, thickness=2):
    cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, scale, (0,0,0), thickness+3, cv2.LINE_AA)
    cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, scale, (255,255,255), thickness, cv2.LINE_AA)

class RepetitionCounter:
    def __init__(self):
        self.pushup_cnt = 0; self.pushup_state = "up"
        self.squat_cnt  = 0; self.squat_state  = "top"
        self.jj_cnt     = 0; self.jj_state     = "closed"

    def update_pushup(self, k):
        needed = [5,6,7,8,9,10,11,12,15,16]
        if not all(is_visible(k, i) for i in needed):
            return
        L = angle(get_kpt_xy(k, 5), get_kpt_xy(k, 7), get_kpt_xy(k, 9))
        R = angle(get_kpt_xy(k, 6), get_kpt_xy(k, 8), get_kpt_xy(k, 10))
        elbow = (L + R) / 2.0
        hipL = angle(get_kpt_xy(k, 5), get_kpt_xy(k, 11), get_kpt_xy(k, 15))
        hipR = angle(get_kpt_xy(k, 6), get_kpt_xy(k, 12), get_kpt_xy(k, 16))
        plank = (hipL > 155 and hipR > 155)
        down_thr, up_thr = 90, 150
        if plank:
            if self.pushup_state == "up" and elbow < down_thr:
                self.pushup_state = "down"
            elif self.pushup_state == "down" and elbow > up_thr:
                self.pushup_cnt += 1
                self.pushup_state = "up"

    def update_squat(self, k):
        needed = [11,12,13,14,15,16]
        if not all(is_visible(k, i) for i in needed):
            return
        L = angle(get_kpt_xy(k, 11), get_kpt_xy(k, 13), get_kpt_xy(k, 15))
        R = angle(get_kpt_xy(k, 12), get_kpt_xy(k, 14), get_kpt_xy(k, 16))
        knee = (L + R) / 2.0
        hip_y  = (get_kpt_xy(k, 11)[1] + get_kpt_xy(k, 12)[1]) / 2
        knee_y = (get_kpt_xy(k, 13)[1] + get_kpt_xy(k, 14)[1]) / 2
        depth_ok = hip_y > knee_y - 5
        bottom_thr, top_thr = 90, 160
        if self.squat_state == "top" and knee < bottom_thr and depth_ok:
            self.squat_state = "bottom"
        elif self.squat_state == "bottom" and knee > top_thr:
            self.squat_cnt += 1
            self.squat_state = "top"

    def update_jj(self, k, frame_w):
        needed = [5,6,9,10,15,16,0]
        if not all(is_visible(k, i) for i in needed):
            return
        wristY = (get_kpt_xy(k, 9)[1] + get_kpt_xy(k, 10)[1]) / 2
        headY  = get_kpt_xy(k, 0)[1]
        hands_up = wristY < headY
        ankles_dist = euclid(get_kpt_xy(k, 15), get_kpt_xy(k, 16))
        feet_apart = ankles_dist > 0.25 * frame_w  # sederhana, pakai lebar frame
        if self.jj_state == "closed" and hands_up and feet_apart:
            self.jj_state = "open"
        elif self.jj_state == "open" and (not hands_up or not feet_apart):
            self.jj_cnt += 1
            self.jj_state = "closed"

def is_standing(kpts):
    head_y = get_kpt_xy(kpts, 0)[1]
    ankle_y = (get_kpt_xy(kpts,15)[1] + get_kpt_xy(kpts,16)[1]) / 2
    return (ankle_y - head_y) > 0.5 * max(1.0, ankle_y)

In [8]:
def infer(frame_rgb, state: RepetitionCounter | None, imgsz=640, conf=0.25):
    # frame dari Gradio = RGB np.uint8 (H,W,3)
    if frame_rgb is None:
        # kembalikan canvas kosong saat belum ada frame
        blank = np.zeros((480, 640, 3), dtype=np.uint8)
        return blank, (state or RepetitionCounter())

    frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
    counter = state or RepetitionCounter()

    with torch.inference_mode():
        res = model.predict(source=frame_bgr, imgsz=imgsz, conf=conf,
                            device=0 if torch.cuda.is_available() else 'cpu',
                            verbose=False)[0]

    kp_obj = res.keypoints
    if kp_obj is not None and kp_obj.data is not None and kp_obj.data.shape[0] > 0:
        n = kp_obj.data.shape[0]
        if res.boxes is not None and getattr(res.boxes, "conf", None) is not None:
            scores = res.boxes.conf.detach().cpu().numpy()
        else:
            scores = np.ones(n, np.float32)
        best_i = int(np.argmax(scores))
        kpts = kp_obj.data[best_i].detach().cpu().numpy().reshape(-1,3)

        H, W = frame_bgr.shape[:2]
        if is_standing(kpts):
            counter.update_jj(kpts, frame_w=W)
        else:
            counter.update_pushup(kpts)
            counter.update_squat(kpts)

        frame_bgr = res.plot()

    # overlay teks counter
    draw_text(frame_bgr, f"Push-up: {counter.pushup_cnt}", 20, 40)
    draw_text(frame_bgr, f"Squat  : {counter.squat_cnt}", 20, 80)
    draw_text(frame_bgr, f"J. Jack: {counter.jj_cnt}",   20, 120)

    out_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    return out_rgb, counter

In [9]:
with gr.Blocks(title="YOLOv8 Pose Realtime - Gradio") as app:
    gr.Markdown("### YOLOv8 Pose Realtime (Webcam) — Counter Push-up / Squat / Jumping Jack")
    with gr.Row():
        cam = gr.Image(sources=["webcam"], streaming=True, label="Webcam")
        out = gr.Image(label="Output")
    state = gr.State()

    # jalankan infer setiap frame webcam (streaming=True)
    cam.stream(fn=infer, inputs=[cam, state], outputs=[out, state])

# Jalankan: ubah share=True jika di Colab / butuh URL publik
app.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>